<a href="https://colab.research.google.com/github/Tanisha1011/BEFL/blob/main/Work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import hashlib
import json
from time import time

# Example dataset
class ExampleDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

def train_local_model(model, train_loader, criterion, optimizer):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    return model.state_dict(), loss.item()

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def federated_aggregation(global_model, local_models):
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([local_models[i][k].float() for i in range(len(local_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    return global_model

class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.new_block(previous_hash='1', proof=100)

    def new_block(self, proof, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transactions = []
        self.chain.append(block)
        return block

    def new_transaction(self, client_id, model_state):
        self.current_transactions.append({
            'client_id': client_id,
            'model_state': model_state,
        })
        return self.last_block['index'] + 1

    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    @property
    def last_block(self):
        return self.chain[-1]

    def proof_of_work(self, last_proof):
        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
        return proof

    @staticmethod
    def valid_proof(last_proof, proof):
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

# Generate synthetic data
np.random.seed(0)
data = np.random.rand(1000, 2)
labels = (data[:, 0] + data[:, 1] > 1).astype(float).reshape(-1, 1)

# Split data into training and testing
train_data, test_data = data[:800], data[800:]
train_labels, test_labels = labels[:800], labels[800:]

# Create dataloaders
train_dataset = ExampleDataset(torch.tensor(train_data, dtype=torch.float32), torch.tensor(train_labels, dtype=torch.float32))
test_dataset = ExampleDataset(torch.tensor(test_data, dtype=torch.float32), torch.tensor(test_labels, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize global model
global_model = SimpleNN()
criterion = nn.BCELoss()
initial_lr = 0.01

# Initialize blockchain
blockchain = Blockchain()

# Federated learning simulation with blockchain
num_rounds = 25
num_clients = 10

for round in range(num_rounds):
    local_models = []
    for client in range(num_clients):
        local_model = SimpleNN()
        local_model.load_state_dict(global_model.state_dict())
        optimizer = optim.SGD(local_model.parameters(), lr=initial_lr)
        local_state, local_loss = train_local_model(local_model, train_loader, criterion, optimizer)

        # Serialize model state to JSON compatible format
        model_state_dict = {k: v.tolist() for k, v in local_state.items()}

        # Record the model update on the blockchain
        blockchain.new_transaction(client_id=client, model_state=model_state_dict)

        local_models.append(local_state)

    # Perform federated aggregation
    global_model = federated_aggregation(global_model, local_models)

    # Create a new block in the blockchain after aggregation
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)
    previous_hash = blockchain.hash(last_block)
    blockchain.new_block(proof, previous_hash)

    # Evaluate the global model
    accuracy = evaluate_model(global_model, test_loader)
    print(f"Round {round + 1}: Global Model Accuracy: {accuracy:.2f}%")

# Display final accuracy
final_accuracy = evaluate_model(global_model, test_loader)
print(f"Final Global Model Accuracy: {final_accuracy:.2f}%")

# Display blockchain
for block in blockchain.chain:
    print(f"Block {block['index']}: {block}")
import matplotlib.pyplot as plt

# Initialize lists to store metrics
global_model_accuracies = []
global_model_losses = []
avg_local_losses = []
num_transactions = []
blockchain_size = []

# Federated learning simulation with blockchain
num_rounds = 25
num_clients = 10

for round in range(num_rounds):
    local_models = []
    local_losses = []
    for client in range(num_clients):
        local_model = SimpleNN()
        local_model.load_state_dict(global_model.state_dict())
        optimizer = optim.SGD(local_model.parameters(), lr=initial_lr)
        local_state, local_loss = train_local_model(local_model, train_loader, criterion, optimizer)

        local_models.append(local_state)
        local_losses.append(local_loss)

        blockchain.new_transaction(client_id=client, model_state={k: v.tolist() for k, v in local_state.items()})

    # Compute metrics for the round
    global_model_accuracies.append(evaluate_model(global_model, test_loader))
    global_model_losses.append(criterion(global_model(torch.tensor(test_data, dtype=torch.float32)), torch.tensor(test_labels, dtype=torch.float32)).item())
    avg_local_losses.append(np.mean(local_losses))
    num_transactions.append(len(blockchain.current_transactions))
    blockchain_size.append(len(blockchain.chain))

    # Perform federated aggregation
    global_model = federated_aggregation(global_model, local_models)

    # Create a new block in the blockchain after aggregation
    last_block = blockchain.last_block
    proof = blockchain.proof_of_work(last_block['proof'])
    previous_hash = blockchain.hash(last_block)
    blockchain.new_block(proof, previous_hash)

    # Print current round's metrics
    print(f"Round {round + 1}: Global Model Accuracy: {global_model_accuracies[-1]:.2f}%, Global Model Loss: {global_model_losses[-1]:.4f}, Average Local Loss: {avg_local_losses[-1]:.4f}")

# Display final accuracy
final_accuracy = evaluate_model(global_model, test_loader)
print(f"Final Global Model Accuracy: {final_accuracy:.2f}%")

# Display blockchain
for block in blockchain.chain:
    print(f"Block {block['index']}: {block}")

# Plotting the metrics
rounds = range(1, num_rounds + 1)

plt.figure(figsize=(14, 10))

# Plot Global Model Accuracy
plt.subplot(3, 2, 1)
plt.plot(rounds, global_model_accuracies, marker='o')
plt.title('Global Model Accuracy')
plt.xlabel('Round')
plt.ylabel('Accuracy (%)')

# Plot Global Model Loss
plt.subplot(3, 2, 2)
plt.plot(rounds, global_model_losses, marker='o')
plt.title('Global Model Loss')
plt.xlabel('Round')
plt.ylabel('Loss')

# Plot Average Local Loss
plt.subplot(3, 2, 3)
plt.plot(rounds, avg_local_losses, marker='o')
plt.title('Average Local Model Loss')
plt.xlabel('Round')
plt.ylabel('Loss')

# Plot Number of Transactions per Round
plt.subplot(3, 2, 4)
plt.plot(rounds, num_transactions, marker='o')
plt.title('Number of Transactions')
plt.xlabel('Round')
plt.ylabel('Number of Transactions')

# Plot Blockchain Size (Number of Blocks)
plt.subplot(3, 2, 5)
plt.plot(rounds, blockchain_size, marker='o')
plt.title('Blockchain Size')
plt.xlabel('Round')
plt.ylabel('Number of Blocks')

plt.tight_layout()
plt.show()
